<a href="https://colab.research.google.com/github/AtharKharal/Data_science_with_python/blob/dev-branch/featuretools_and_TPOT_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import featuretools as ft
import sklearn as sk

In [3]:
es = ft.EntitySet("mydata")

# Create an entity from the client dataframe
# This dataframe already has an index and a time index col namely 'dt_time_col'
es = es.entity_from_dataframe(entity_id = 'my_df_id', dataframe = my_df, 
                              index = 'idx_of_my_df', time_index = 'dt_time_col')


# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='my_df_id', 
                                 max_depth = 2)

'0.4.1'

In [2]:
ft.__version__

'0.4.1'